In [1]:
import matplotlib, os
import numpy as np
import imageio.v3 as imageio
from IPython.display import clear_output
from shared.Render import collect_saved_frames, render_function_point
from shared.TestFunctions import TEST_FUNCTIONS
matplotlib.use('Agg')
np.random.seed(42)

#### The SOMA AllToOne algorithm

In [2]:
def soma_all_to_one(func, rnge, pop_size=30, prt=0.4, path_length=3.0, step=0.11, migrations=10):
    # Population initialization and fitness evaluation
    positions = np.random.uniform(rnge[0], rnge[1], size=(pop_size, 2))
    fitness = np.apply_along_axis(lambda pos: func(tuple(pos)), 1, positions)
    gbest_index = np.argmin(fitness)
    gbest_position = positions[gbest_index].copy()
    gbest_value = fitness[gbest_index]
    iteration = 0
    yield iteration, tuple(gbest_position)
    # Migration loop
    for _ in range(migrations):
        gbest_index = np.argmin(fitness)
        leader_position = positions[gbest_index].copy()
        # Move each non-leader individual toward the leader
        for i in range(pop_size):
            if i == gbest_index:
                continue
            start_position = positions[i].copy()
            best_local_pos = start_position.copy()
            best_local_val = fitness[i]
            # Sample PRT vector once per individual (canonical behavior)
            prt_vector = (np.random.uniform(size=2) < prt).astype(float)
            if not prt_vector.any():
                prt_vector[np.random.randint(2)] = 1.0
            # Step along the path toward the leader
            for t in np.arange(step, path_length + step, step):
                candidate = start_position + (leader_position - start_position) * t * prt_vector
                candidate = np.clip(candidate, rnge[0], rnge[1])
                value = func(tuple(candidate))
                if value < best_local_val:
                    best_local_val = value
                    best_local_pos = candidate.copy()
                if value < gbest_value:
                    gbest_value = value
                    gbest_position = candidate.copy()
                    iteration += 1
                    yield iteration, tuple(gbest_position)
            positions[i] = best_local_pos
            fitness[i] = best_local_val
        # End-of-migration global best check
        gbest_index = np.argmin(fitness)
        if fitness[gbest_index] < gbest_value:
            gbest_value = fitness[gbest_index]
            gbest_position = positions[gbest_index].copy()
            iteration += 1
            yield iteration, tuple(gbest_position)

#### Run the SOMA AllToOne algorithm and collect frames for multiple functions

In [3]:
gif_folder_path = "gifs/soma_alltoone"
os.makedirs(gif_folder_path, exist_ok=True)

In [4]:
for func, params in TEST_FUNCTIONS.items():
    print(f"Test function: {func.__name__}")
    rnge = (params[0], params[1])
    resolution = params[2]
    for idx, point in soma_all_to_one(func, rnge):
        frame = render_function_point(func, rnge=rnge, resolution=resolution, point=point, iteration=idx)
        imageio.imwrite(os.path.join(gif_folder_path, f"{func.__name__}{idx:08d}.png"), frame)
    collect_saved_frames(gif_folder_path, f"{str.capitalize(func.__name__)}.gif", fps=5)
clear_output()

#### Visualize the SOMA AllToOne algorithm on multiple functions

![Sphere](gifs/soma_alltoone/Sphere.gif)
![Ackley](gifs/soma_alltoone/Ackley.gif)
![Rastrigin](gifs/soma_alltoone/Rastrigin.gif)
![Rosenbrock](gifs/soma_alltoone/Rosenbrock.gif)
![Griewank](gifs/soma_alltoone/Griewank.gif)
![Schwefel](gifs/soma_alltoone/Schwefel.gif)
![Levy](gifs/soma_alltoone/Levy.gif)
![Michalewicz](gifs/soma_alltoone/Michalewicz.gif)
![Zakharov](gifs/soma_alltoone/Zakharov.gif)